In [1]:
# *EXECUTE*
import requests, time, os
import pandas as pd

from bs4 import BeautifulSoup

In [ ]:
# Query data
# *EXECUTE*

responses = []
page_count = 65
for i in range(page_count):
    res = requests.get("https://steamspy.com/api.php?request=all&page=" + str(i))
    if res is not None and res.ok:
        responses.append(res.json())
        print("Found page", i)
    else:
        print("Error on request with page id", i)
        responses.append({})
    
    if i != page_count - 1:
        time.sleep(70)
ldict = []
for r in responses:
    for k in r.keys():
        ldict.append(r[k])

df = pd.DataFrame(ldict)
df.to_csv("data/raw.csv", index=False)
print(df)

In [ ]:
# Merge data; I had a bug that interrupted my downloads.

d1, d2, d3, d4 = pd.read_csv("data/raw13.csv"), pd.read_csv("data/raw49.csv"), pd.read_csv("data/raw59.csv"), pd.read_csv("data/raw61.csv")
fin = pd.concat([d1, d2, d3, d4])
fin = fin.reset_index()
fin.to_csv("data/raw.csv", index=False)

In [ ]:
# Create extra columns and clean
# *EXECUTE*

df = pd.read_csv("data/steam_games.csv")
downloaded_files = [int(i.split(".")[0]) for i in os.listdir("data/descriptions")]

ids, total_revs, desc_downloaded = [], [], []
for url in df.url:
    url = url.split("/")
    if len(url) <= 4 or len(url[4]) == 0:
        ids.append(-1)
    else:
        ids.append(int(url[4]))
    if ids[-1] in downloaded_files:
        desc_downloaded.append(True)
    else:
        desc_downloaded.append(False)
        
for all_reviews in df.all_reviews:
    try:
        all_reviews = all_reviews[all_reviews.index("(") + 1:all_reviews.index(")")]
        total_revs.append(int(all_reviews.replace(",", "")))
    except:
        total_revs.append(-1)

df["appid"] = ids
df["total_reviews"] = total_revs
df["downloaded"] = desc_downloaded
df = df[df.appid>-1][df.total_reviews>-1]
df = df.sample(frac=1).reset_index()
print(df)

In [52]:
# Download pages
# *EXECUTE*

for i, r in df.iterrows():
    if i % 10 == 0:
        if not r.downloaded:
            df.to_csv("data/partial.csv", index=False)
    if r.types != "app" or r.downloaded:
        continue
    # Make request
    print("Redownloading", i)
    try:
        res = requests.get(r.url)
        if res is not None and res.ok:
            res = res.content.decode('utf-8')
            with open("data/descriptions/" + str(r.appid) + '.txt', 'w', encoding='utf-8') as f:
                f.write(res)
                df.at[i, "downloaded"] = True
        else:
            print("Error at index", i)
    except:
        print("Error at index", i)
        df.at[i, "downloaded"] = False
        time.sleep(10)
    time.sleep(2)

In [ ]:
# Extract game description
# *EXECUTE*
df = pd.read_csv("data/partial.csv")
descriptions = []
errors = []
for i, r in df.iterrows():
    if i % 100 == 0:
        print(i, "/", len(df))
    contents = ""
    with open("data/descriptions/" + str(r.appid) + '.txt', 'r', encoding='utf-8') as f:
        contents = f.read()
    
    soup = BeautifulSoup(contents).select("#game_area_description")
    if len(soup) == 0:
        descriptions.append("")
        errors.append(r.appid)
        if len(errors) % 10 == 0:
            print("At", len(errors), "errors.")
        df.at[i, "downloaded"] = False
    else:
        descriptions.append(soup[0].text)

df["full_description"] = descriptions
df.to_csv("data/dataset_draft.csv", index=False)
print("Finished extracting game descriptions;", len(errors), "total errors.")

In [14]:
print(df.columns)
df = df.drop(columns=["desc_snippet", "achievements"])

Index(['index', 'url', 'types', 'name', 'desc_snippet', 'recent_reviews',
       'all_reviews', 'release_date', 'developer', 'publisher', 'popular_tags',
       'game_details', 'languages', 'achievements', 'genre',
       'game_description', 'mature_content', 'minimum_requirements',
       'recommended_requirements', 'original_price', 'discount_price', 'appid',
       'total_reviews', 'downloaded', 'full_description'],
      dtype='object')


In [28]:
print(df.iloc[7].full_description)
print(df.iloc[7].game_description)


About This Game

							Don’t miss the single-player campaign from the game that IGN awarded Best Xbox One Game of 2014, the game that Polygon rated 9 out of 10, and the game that Eurogamer calls “a breath of fresh air.” In Sunset Overdrive, the year is 2027 and Sunset City is under siege. A contaminated energy drink has transformed most of the population into toxic mutants. For many it's the end of the world, but for you it’s a dream come true. Your old boss? Dead. Your boring job? Gone. Transform the open-world into your tactical playground by grinding, vaulting and wall-running across the city while using a devastating, unconventional arsenal. With hyper-agility, unique weapons, and customizable special abilities, Sunset Overdrive rewrites the rules of traditional shooters and delivers an explosive, irreverent, stylish, and totally unique adventure.


Embrace the chaos of Sunset City through a hyper-colorful, post-apocalyptic single-player campaign and two bonus expansions, Mystery